In [41]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import mysql.connector
import pandas as pd


In [42]:
def connect_to_db():
    return mysql.connector.connect(
       host="localhost",
       user="root",
       password="",
       database="ems"
    )

In [43]:
def run_query():
    try:
        conn = connect_to_db()
        cursor = conn.cursor(dictionary=True)

        query = """
            SELECT 
                e.UserID,
                e.empname,
                e.empdoj,
                COUNT(*) AS present_days
            FROM (
                SELECT 
                    c.UserID,
                    DATE(c.AttendanceTime) AS att_date
                FROM 
                    CamsBiometricAttendance c
                WHERE 
                    c.AttendanceType IN ('CheckIn', 'CheckOut')
                GROUP BY 
                    c.UserID, DATE(c.AttendanceTime)
                HAVING 
                    COUNT(DISTINCT c.AttendanceType) = 2
            ) valid_days
            JOIN emp e ON e.UserID = valid_days.UserID
            WHERE 
                valid_days.att_date >= e.empdoj
                AND valid_days.att_date <= CURDATE()
            GROUP BY 
                e.UserID, e.empname, e.empdoj;
        """
        cursor.execute(query)
        result = cursor.fetchall()

        # Close the connection
        conn.close()

        # Convert the result to a DataFrame for tabular display
        df = pd.DataFrame(result)

        return df

    except mysql.connector.Error as err:
        # Handle database connection or query errors
        print(f"Database error: {err}")
        return []

# Run the query and display the output in tabular format
run_query()

,UserID,empname,empdoj,present_days
0,1,YATHIRAJAM SOWMYA,2022-10-01,325
1,11,ARSHPREET KAUR,2024-07-29,179
2,11,SANDIP BERA,2023-07-25,334
3,12,GANDI MANOHARA RAO,2022-11-01,122
4,13,SHANKAR,2024-07-30,179
...,...,...,...,...
70,7,RAHUL,2024-06-10,211
71,8,BAGADI PRAKASH RAO,2022-10-01,169
72,8,FULKURIA INDOWAR,2024-06-03,155
73,9,DEEPAK YADAV,2024-07-13,172


In [44]:
# def run_query():
#     try:
#         conn = connect_to_db()
#         cursor = conn.cursor(dictionary=True)

#         query = """
#            WITH RECURSIVE all_dates AS (
#     -- Generate a series of all dates starting from the earliest employee joining date
#     SELECT (SELECT MIN(empdoj) FROM emp) AS dt
#     UNION ALL
#     SELECT dt + INTERVAL 1 DAY
#     FROM all_dates
#     WHERE dt + INTERVAL 1 DAY <= '2025-03-31'  -- End at March 31, 2025
# ),
# valid_dates AS (
#     -- Filter out Sundays and holidays
#     SELECT dt
#     FROM all_dates
#     WHERE 
#         DAYOFWEEK(dt) != 1  -- Exclude Sundays
#         AND dt NOT IN (SELECT `date` FROM holiday)  -- Exclude holidays
# ),
# working_days_per_employee AS (
#     -- Calculate working days for each unique combination of UserID and empname
#     SELECT 
#         e.UserID,
#         e.empname,
#         COUNT(*) AS working_days
#     FROM emp e
#     JOIN valid_dates v ON v.dt BETWEEN e.empdoj AND '2025-03-31'  -- Ensure within the joining date range and up to March 31, 2025
#     GROUP BY e.UserID, e.empname
# ),
# present_days AS (
#     -- Select present days based on check-in and check-out
#     SELECT 
#         c.UserID,
#         DATE(c.AttendanceTime) AS att_date
#     FROM 
#         CamsBiometricAttendance c
#     WHERE 
#         c.AttendanceType IN ('CheckIn', 'CheckOut')
#     GROUP BY 
#         c.UserID, DATE(c.AttendanceTime)
#     HAVING 
#         COUNT(DISTINCT c.AttendanceType) = 2
# ),
# present_days_per_employee AS (
#     -- Count present days per unique combination of UserID and empname
#     SELECT 
#         e.UserID,
#         e.empname,
#         COUNT(*) AS present_days
#     FROM 
#         emp e
#     JOIN present_days p ON e.UserID = p.UserID
#     WHERE 
#         p.att_date >= e.empdoj
#         AND p.att_date <= '2025-03-31'  -- Consider attendance up to March 31, 2025
#     GROUP BY e.UserID, e.empname
# )
# SELECT 
#     e.UserID,
#     e.empname,
#     e.empdoj,
#     COALESCE(p.present_days, 0) AS present_days,
#     COALESCE(w.working_days, 0) AS working_days
# FROM emp e
# LEFT JOIN present_days_per_employee p ON e.UserID = p.UserID AND e.empname = p.empname
# LEFT JOIN working_days_per_employee w ON e.UserID = w.UserID AND e.empname = w.empname;

#         """

#         cursor.execute(query)
#         result = cursor.fetchall()

#         conn.close()

#         df = pd.DataFrame(result)

#         return df

#     except mysql.connector.Error as err:
#         print(f"Database error: {err}")
#         return []
# run_query()

In [45]:
import pandas as pd
import mysql.connector


def first_feature():
    try:
        conn = connect_to_db()
        cursor = conn.cursor(dictionary=True)

        query = """
WITH RECURSIVE all_dates AS (
    SELECT DATE('2024-01-01') AS dt
    UNION ALL
    SELECT dt + INTERVAL 1 DAY
    FROM all_dates
    WHERE dt + INTERVAL 1 DAY <= '2025-03-31'
),
valid_dates_vskp AS (
    SELECT dt
    FROM all_dates
    WHERE 
        DAYOFWEEK(dt) != 1
        AND dt NOT IN (SELECT `date` FROM holiday)
),
valid_dates_ggm AS (
    SELECT dt
    FROM all_dates
    WHERE 
        DAYOFWEEK(dt) != 1
        AND dt NOT IN (SELECT `date` FROM holiday_ggm)
),
present_days_vskp AS (
    SELECT 
        c.UserID,
        DATE(c.AttendanceTime) AS att_date
    FROM 
        CamsBiometricAttendance c
    WHERE 
        c.AttendanceType IN ('CheckIn', 'CheckOut')
    GROUP BY 
        c.UserID, DATE(c.AttendanceTime)
    HAVING 
        COUNT(DISTINCT c.AttendanceType) = 2
),
present_days_ggm AS (
    SELECT 
        c.UserID,
        DATE(c.AttendanceTime) AS att_date
    FROM 
        CamsBiometricAttendance_GGM c
    WHERE 
        c.AttendanceType IN ('CheckIn', 'CheckOut')
    GROUP BY 
        c.UserID, DATE(c.AttendanceTime)
    HAVING 
        COUNT(DISTINCT c.AttendanceType) = 2
),
working_days_per_employee AS (
    SELECT 
        e.UserID,
        e.empname,
        e.work_location,
        CASE 
            WHEN e.work_location = 'Visakhapatnam' THEN 
                (SELECT COUNT(*) FROM valid_dates_vskp v 
                 WHERE v.dt BETWEEN GREATEST(e.empdoj, '2024-01-01') AND '2025-03-31')
            WHEN e.work_location = 'Gurugram' THEN 
                (SELECT COUNT(*) FROM valid_dates_ggm v 
                 WHERE v.dt BETWEEN GREATEST(e.empdoj, '2024-01-01') AND '2025-03-31')
            ELSE 0
        END AS working_days
    FROM emp e
),
present_days_per_employee AS (
    SELECT 
        e.UserID,
        e.empname,
        e.work_location,
        CASE 
            WHEN e.work_location = 'Visakhapatnam' THEN 
                (SELECT COUNT(*) 
                 FROM present_days_vskp p 
                 WHERE p.UserID = e.UserID 
                 AND p.att_date BETWEEN GREATEST(e.empdoj, '2024-01-01') AND '2025-03-31')
            WHEN e.work_location = 'Gurugram' THEN 
                (SELECT COUNT(*) 
                 FROM present_days_ggm p 
                 WHERE p.UserID = e.UserID 
                 AND p.att_date BETWEEN GREATEST(e.empdoj, '2024-01-01') AND '2025-03-31')
            ELSE 0
        END AS present_days
    FROM emp e
)

SELECT 
    e.UserID,
    e.empname,
    e.empdoj,
    e.work_location,
    COALESCE(p.present_days, 0) AS present_days,
    COALESCE(w.working_days, 0) AS working_days
FROM emp e
LEFT JOIN present_days_per_employee p ON e.UserID = p.UserID AND e.empname = p.empname
LEFT JOIN working_days_per_employee w ON e.UserID = w.UserID AND e.empname = w.empname;
        """

        cursor.execute(query)
        result = cursor.fetchall()

        conn.close()

        df = pd.DataFrame(result)
        return df

    except mysql.connector.Error as err:
        print(f"Database error: {err}")
        return []

df = first_feature()
df.head(25)


,UserID,empname,empdoj,work_location,present_days,working_days
0,7,BONDI MANOHAR,2023-09-11,Visakhapatnam,325,360
1,8,BAGADI PRAKASH RAO,2022-10-01,Visakhapatnam,169,360
2,1,YATHIRAJAM SOWMYA,2022-10-01,Visakhapatnam,325,360
3,16,NAKKA ARAVIND REDDY,2022-12-01,Visakhapatnam,330,360
4,38,GORAPALLI MEGHA SHYAM,2022-12-05,Visakhapatnam,284,360
5,500,ABHILASH KUMAR SAHU,2023-03-01,Visakhapatnam,0,360
6,17,KUNDRAPU SRINIVAS,2023-07-11,Visakhapatnam,330,360
7,25,SURAMPUDI LAXMAN,2023-07-11,Visakhapatnam,332,360
8,19,INDALA SATISH KUMAR,2023-07-11,Visakhapatnam,330,360
9,12,GANDI MANOHARA RAO,2022-11-01,Visakhapatnam,122,360


In [46]:
#mine
import pandas as pd
import mysql.connector


def first_feature():
    try:
        conn = connect_to_db()
        cursor = conn.cursor(dictionary=True)

        query = """
WITH RECURSIVE all_dates AS (
    SELECT DATE('2024-01-01') AS dt
    UNION ALL
    SELECT dt + INTERVAL 1 DAY
    FROM all_dates
    WHERE dt + INTERVAL 1 DAY <= '2025-03-31'
),
valid_dates_vskp AS (
    SELECT dt
    FROM all_dates
    WHERE 
        DAYOFWEEK(dt) != 1
        AND dt NOT IN (SELECT `date` FROM holiday)
),
valid_dates_ggm AS (
    SELECT dt
    FROM all_dates
    WHERE 
        DAYOFWEEK(dt) != 1
        AND dt NOT IN (SELECT `date` FROM holiday_ggm)
),
present_days_vskp AS (
    SELECT 
        c.UserID,
        DATE(c.AttendanceTime) AS att_date
    FROM 
        CamsBiometricAttendance c
    WHERE 
        c.AttendanceType IN ('CheckIn', 'CheckOut')
    GROUP BY 
        c.UserID, DATE(c.AttendanceTime)
    HAVING 
        COUNT(DISTINCT c.AttendanceType) = 2
),
present_days_ggm AS (
    SELECT 
        c.UserID,
        DATE(c.AttendanceTime) AS att_date
    FROM 
        CamsBiometricAttendance_GGM c
    WHERE 
        c.AttendanceType IN ('CheckIn', 'CheckOut')
    GROUP BY 
        c.UserID, DATE(c.AttendanceTime)
    HAVING 
        COUNT(DISTINCT c.AttendanceType) = 2
),
leave_days_per_employee AS (
    SELECT 
        e.UserID,
        SUM(DATEDIFF(l.`to`, l.`from`) + 1) AS leave_days
    FROM emp e
    LEFT JOIN leaves l ON l.empname = e.empname
        AND l.status = 1
        AND l.leavetype IN ('WORK FROM HOME', 'ON DUTY', 'OFFICIAL LEAVE')
    WHERE e.work_location IN ('Visakhapatnam', 'Gurugram')
    GROUP BY e.UserID
),
working_days_per_employee AS (
    SELECT 
        e.UserID,
        e.empname,
        e.work_location,
        CASE 
            WHEN e.work_location = 'Visakhapatnam' THEN 
                (SELECT COUNT(*) FROM valid_dates_vskp v 
                 WHERE v.dt BETWEEN GREATEST(e.empdoj, '2024-01-01') AND '2025-03-31')
            WHEN e.work_location = 'Gurugram' THEN 
                (SELECT COUNT(*) FROM valid_dates_ggm v 
                 WHERE v.dt BETWEEN GREATEST(e.empdoj, '2024-01-01') AND '2025-03-31')
            ELSE 0
        END AS working_days
    FROM emp e
),
present_days_per_employee AS (
    SELECT 
        e.UserID,
        e.empname,
        e.work_location,
        CASE 
            WHEN e.work_location = 'Visakhapatnam' THEN 
                (SELECT COUNT(*) 
                 FROM present_days_vskp p 
                 WHERE p.UserID = e.UserID 
                   AND p.att_date BETWEEN GREATEST(e.empdoj, '2024-01-01') AND '2025-03-31')
            WHEN e.work_location = 'Gurugram' THEN 
                (SELECT COUNT(*) 
                 FROM present_days_ggm p 
                 WHERE p.UserID = e.UserID 
                   AND p.att_date BETWEEN GREATEST(e.empdoj, '2024-01-01') AND '2025-03-31')
            ELSE 0
        END AS present_days_attendance
    FROM emp e
)

SELECT 
    e.UserID,
    e.empname,
    e.empdoj,
    e.work_location,
    COALESCE(p.present_days_attendance, 0) + COALESCE(l.leave_days, 0) AS present_days, 
    COALESCE(w.working_days, 0) AS working_days
FROM emp e
LEFT JOIN present_days_per_employee p ON e.UserID = p.UserID
LEFT JOIN working_days_per_employee w ON e.UserID = w.UserID
LEFT JOIN leave_days_per_employee l ON e.UserID = l.UserID;
        """

        cursor.execute(query)
        result = cursor.fetchall()

        conn.close()

        df = pd.DataFrame(result)
        return df

    except mysql.connector.Error as err:
        print(f"Database error: {err}")
        return []

df = first_feature()
df.head(25)


,UserID,empname,empdoj,work_location,present_days,working_days
0,7,BONDI MANOHAR,2023-09-11,Visakhapatnam,332,360
1,7,RAHUL,2024-06-10,Gurugram,332,360
2,7,BONDI MANOHAR,2023-09-11,Visakhapatnam,114,360
3,7,RAHUL,2024-06-10,Gurugram,114,360
4,8,BAGADI PRAKASH RAO,2022-10-01,Visakhapatnam,345,360
5,8,FULKURIA INDOWAR,2024-06-03,Gurugram,345,360
6,8,BAGADI PRAKASH RAO,2022-10-01,Visakhapatnam,236,360
7,8,FULKURIA INDOWAR,2024-06-03,Gurugram,236,360
8,1,YATHIRAJAM SOWMYA,2022-10-01,Visakhapatnam,328,360
9,16,NAKKA ARAVIND REDDY,2022-12-01,Visakhapatnam,332,360


In [47]:
# #varsh
# import pandas as pd
# import mysql.connector


# def first_feature():
#     try:
#         conn = connect_to_db()
#         cursor = conn.cursor(dictionary=True)

#         query = """

# WITH RECURSIVE all_dates AS (
#     SELECT DATE('2024-01-01') AS dt
#     UNION ALL
#     SELECT dt + INTERVAL 1 DAY
#     FROM all_dates
#     WHERE dt + INTERVAL 1 DAY <= '2025-03-31'
# ),
# valid_dates_vskp AS (
#     SELECT dt
#     FROM all_dates
#     WHERE DAYOFWEEK(dt) != 1
#       AND dt NOT IN (SELECT `date` FROM holiday)
# ),
# valid_dates_ggm AS (
#     SELECT dt
#     FROM all_dates
#     WHERE DAYOFWEEK(dt) != 1
#       AND dt NOT IN (SELECT `date` FROM holiday_ggm)
# ),
# attendance_vskp AS (
#     SELECT 
#         UserID,
#         DATE(AttendanceTime) AS att_date
#     FROM CamsBiometricAttendance
#     WHERE AttendanceType IN ('CheckIn', 'CheckOut')
#     GROUP BY UserID, DATE(AttendanceTime)
#     HAVING COUNT(DISTINCT AttendanceType) = 2
# ),
# attendance_ggm AS (
#     SELECT 
#         UserID,
#         DATE(AttendanceTime) AS att_date
#     FROM CamsBiometricAttendance_GGM
#     WHERE AttendanceType IN ('CheckIn', 'CheckOut')
#     GROUP BY UserID, DATE(AttendanceTime)
#     HAVING COUNT(DISTINCT AttendanceType) = 2
# ),
# leave_days AS (
#     SELECT 
#         e.empname,
#         SUM(DATEDIFF(l.`to`, l.`from`) + 1) AS leave_days
#     FROM emp e
#     LEFT JOIN leaves l 
#         ON l.empname = e.empname
#         AND l.status = 1
#         AND l.leavetype IN ('WORK FROM HOME', 'ON DUTY', 'OFFICIAL LEAVE')
#     WHERE e.work_location IN ('Visakhapatnam', 'Gurugram')
#     GROUP BY e.empname
# ),
# present_days AS (
#     SELECT 
#         e.empname,
#         CASE 
#             WHEN e.work_location = 'Visakhapatnam' THEN (
#                 SELECT COUNT(*)
#                 FROM attendance_vskp a
#                 WHERE a.UserID = e.UserID
#                   AND a.att_date BETWEEN GREATEST(e.empdoj, '2024-01-01') AND '2025-03-31'
#             )
#             WHEN e.work_location = 'Gurugram' THEN (
#                 SELECT COUNT(*)
#                 FROM attendance_ggm a
#                 WHERE a.UserID = e.UserID
#                   AND a.att_date BETWEEN GREATEST(e.empdoj, '2024-01-01') AND '2025-03-31'
#             )
#             ELSE 0
#         END AS biometric_present
#     FROM emp e
# ),
# working_days AS (
#     SELECT 
#         e.empname,
#         CASE 
#             WHEN e.work_location = 'Visakhapatnam' THEN (
#                 SELECT COUNT(*)
#                 FROM valid_dates_vskp v
#                 WHERE v.dt BETWEEN GREATEST(e.empdoj, '2024-01-01') AND '2025-03-31'
#             )
#             WHEN e.work_location = 'Gurugram' THEN (
#                 SELECT COUNT(*)
#                 FROM valid_dates_ggm v
#                 WHERE v.dt BETWEEN GREATEST(e.empdoj, '2024-01-01') AND '2025-03-31'
#             )
#             ELSE 0
#         END AS total_working_days
#     FROM emp e
# )

# SELECT 
#     e.empname,
#     e.empdoj,
#     e.work_location,
#     (COALESCE(p.biometric_present, 0) + COALESCE(l.leave_days, 0)) AS present_days,
#     COALESCE(w.total_working_days, 0) AS working_days
# FROM emp e
# LEFT JOIN present_days p ON e.empname = p.empname
# LEFT JOIN leave_days l ON e.empname = l.empname
# LEFT JOIN working_days w ON e.empname = w.empname;
# """



#         cursor.execute(query)
#         result = cursor.fetchall()

#         conn.close()

#         df = pd.DataFrame(result)
#         return df

#     except mysql.connector.Error as err:
#         print(f"Database error: {err}")
#         return pd.DataFrame() 

# df = first_feature()
# df.head(25)


In [48]:
import pandas as pd
import mysql.connector



def first_feature():
    try:
        conn = connect_to_db()
        cursor = conn.cursor(dictionary=True)

        query = """
       WITH RECURSIVE all_dates AS (
    SELECT DATE('2024-01-01') AS dt
    UNION ALL
    SELECT dt + INTERVAL 1 DAY
    FROM all_dates
    WHERE dt + INTERVAL 1 DAY <= '2025-03-31'
),
valid_dates_vskp AS (
    SELECT dt
    FROM all_dates
    WHERE DAYOFWEEK(dt) != 1  -- Excluding Sundays
      AND dt NOT IN (SELECT `date` FROM holiday)
),
valid_dates_ggm AS (
    SELECT dt
    FROM all_dates
    WHERE DAYOFWEEK(dt) != 1  -- Excluding Sundays
      AND dt NOT IN (SELECT `date` FROM holiday_ggm)
),
attendance_vskp AS (
    SELECT 
        UserID,
        DATE(AttendanceTime) AS att_date
    FROM CamsBiometricAttendance
    WHERE AttendanceType IN ('CheckIn', 'CheckOut')
    GROUP BY UserID, DATE(AttendanceTime)
    HAVING COUNT(DISTINCT AttendanceType) = 2
),
attendance_ggm AS (
    SELECT 
        UserID,
        DATE(AttendanceTime) AS att_date
    FROM CamsBiometricAttendance_GGM
    WHERE AttendanceType IN ('CheckIn', 'CheckOut')
    GROUP BY UserID, DATE(AttendanceTime)
    HAVING COUNT(DISTINCT AttendanceType) = 2
),
leave_days AS (
    SELECT 
        e.empname,
        SUM(TIMESTAMPDIFF(DAY, l.`from`, l.`to`)) AS leave_days  -- Calculate the number of days between 'from' and 'to'
    FROM emp e
    LEFT JOIN leaves l 
        ON l.empname = e.empname
        AND l.status = 1
        AND l.leavetype IN ('WORK FROM HOME', 'ON DUTY', 'OFFICIAL LEAVE')
    WHERE e.work_location IN ('Visakhapatnam', 'Gurugram')
    GROUP BY e.empname
),
present_days AS (
    SELECT 
        e.empname,
        CASE 
            WHEN e.work_location = 'Visakhapatnam' THEN (
                SELECT COUNT(*)
                FROM attendance_vskp a
                WHERE a.UserID = e.UserID
                  AND a.att_date BETWEEN GREATEST(e.empdoj, '2024-01-01') AND '2025-03-31'
            )
            WHEN e.work_location = 'Gurugram' THEN (
                SELECT COUNT(*)
                FROM attendance_ggm a
                WHERE a.UserID = e.UserID
                  AND a.att_date BETWEEN GREATEST(e.empdoj, '2024-01-01') AND '2025-03-31'
            )
            ELSE 0
        END AS biometric_present
    FROM emp e
),
working_days AS (
    SELECT 
        e.empname,
        CASE 
            WHEN e.work_location = 'Visakhapatnam' THEN (
                SELECT COUNT(*)
                FROM valid_dates_vskp v
                WHERE v.dt BETWEEN GREATEST(e.empdoj, '2024-01-01') AND '2025-03-31'
            )
            WHEN e.work_location = 'Gurugram' THEN (
                SELECT COUNT(*)
                FROM valid_dates_ggm v
                WHERE v.dt BETWEEN GREATEST(e.empdoj, '2024-01-01') AND '2025-03-31'
            )
            ELSE 0
        END AS total_working_days
    FROM emp e
)

SELECT 
    e.empname,
    e.empdoj,
    e.work_location,
    (COALESCE(p.biometric_present, 0) + COALESCE(l.leave_days, 0)) AS present_days,
    COALESCE(w.total_working_days, 0) AS working_days
FROM emp e
LEFT JOIN present_days p ON e.empname = p.empname
LEFT JOIN leave_days l ON e.empname = l.empname
LEFT JOIN working_days w ON e.empname = w.empname;

        """

        cursor.execute(query)
        result = cursor.fetchall()

        conn.close()

        # Convert the result into a DataFrame for easy handling
        df = pd.DataFrame(result)
        return df

    except mysql.connector.Error as err:
        print(f"Database error: {err}")
        return pd.DataFrame()

# Fetch and display the results
df = first_feature()
df.head(25)


,empname,empdoj,work_location,present_days,working_days
0,BONDI MANOHAR,2023-09-11,Visakhapatnam,325,360
1,BAGADI PRAKASH RAO,2022-10-01,Visakhapatnam,332,360
2,YATHIRAJAM SOWMYA,2022-10-01,Visakhapatnam,325,360
3,NAKKA ARAVIND REDDY,2022-12-01,Visakhapatnam,330,360
4,GORAPALLI MEGHA SHYAM,2022-12-05,Visakhapatnam,349,360
5,ABHILASH KUMAR SAHU,2023-03-01,Visakhapatnam,0,360
6,KUNDRAPU SRINIVAS,2023-07-11,Visakhapatnam,330,360
7,SURAMPUDI LAXMAN,2023-07-11,Visakhapatnam,332,360
8,INDALA SATISH KUMAR,2023-07-11,Visakhapatnam,330,360
9,GANDI MANOHARA RAO,2022-11-01,Visakhapatnam,122,360


In [49]:
df['empname'] = df['empname']
df_user_49 = df[df['empname'] == "GUJJI VARSHITHA"]
print(df_user_49)


Empty DataFrame
Columns: [empname, empdoj, work_location, present_days, working_days]
Index: []


In [50]:
#varsh
import pandas as pd
import mysql.connector


def first_feature():
    try:
        conn = connect_to_db()
        cursor = conn.cursor(dictionary=True)

        query = """
 WITH RECURSIVE all_dates AS (
    SELECT DATE('2024-01-01') AS dt
    UNION ALL
    SELECT dt + INTERVAL 1 DAY
    FROM all_dates
    WHERE dt + INTERVAL 1 DAY <= '2025-03-31'
),
attendance_vskp AS (
    SELECT 
        UserID,
        DATE(AttendanceTime) AS att_date
    FROM CamsBiometricAttendance
    WHERE AttendanceType IN ('CheckIn', 'CheckOut')
    GROUP BY UserID, DATE(AttendanceTime)
    HAVING COUNT(DISTINCT AttendanceType) = 2
),
attendance_ggm AS (
    SELECT 
        UserID,
        DATE(AttendanceTime) AS att_date
    FROM CamsBiometricAttendance_GGM
    WHERE AttendanceType IN ('CheckIn', 'CheckOut')
    GROUP BY UserID, DATE(AttendanceTime)
    HAVING COUNT(DISTINCT AttendanceType) = 2
),
present_days AS (
    SELECT 
        e.empname,
        e.UserID,
        CASE 
            WHEN e.work_location = 'Visakhapatnam' THEN (
                SELECT GROUP_CONCAT(DISTINCT a.att_date ORDER BY a.att_date)
                FROM attendance_vskp a
                WHERE a.UserID = e.UserID
                  AND a.att_date BETWEEN GREATEST(e.empdoj, '2024-01-01') AND '2025-03-31'
            )
            WHEN e.work_location = 'Gurugram' THEN (
                SELECT GROUP_CONCAT(DISTINCT a.att_date ORDER BY a.att_date)
                FROM attendance_ggm a
                WHERE a.UserID = e.UserID
                  AND a.att_date BETWEEN GREATEST(e.empdoj, '2024-01-01') AND '2025-03-31'
            )
            ELSE ''
        END AS present_dates
    FROM emp e
    WHERE e.empname = 'PUTSALA HARSHA PRIYA'
)

SELECT 
    e.empname,
    e.UserID,
    p.present_dates
FROM emp e
LEFT JOIN present_days p ON e.UserID = p.UserID
WHERE e.empname = 'PUTSALA HARSHA PRIYA';


        """
        cursor.execute(query)
        result = cursor.fetchall()

        conn.close()

        df = pd.DataFrame(result)
        return df

    except mysql.connector.Error as err:
        print(f"Database error: {err}")
        return pd.DataFrame() 

df = first_feature()
print(df)


                empname UserID  \
0  PUTSALA HARSHA PRIYA     50   

                                       present_dates  
0  2024-06-04,2024-06-05,2024-06-06,2024-06-07,20...  


In [51]:
import pandas as pd
import mysql.connector


# Function to fetch present days
def fetch_present_days():
    try:
        conn = connect_to_db()
        cursor = conn.cursor(dictionary=True)

        query = """
        WITH RECURSIVE all_dates AS (
            SELECT DATE('2024-01-01') AS dt
            UNION ALL
            SELECT dt + INTERVAL 1 DAY
            FROM all_dates
            WHERE dt + INTERVAL 1 DAY <= '2025-03-31'
        ),
        attendance_vskp AS (
            SELECT 
                UserID,
                DATE(AttendanceTime) AS att_date
            FROM CamsBiometricAttendance
            WHERE AttendanceType IN ('CheckIn', 'CheckOut')
            GROUP BY UserID, DATE(AttendanceTime)
            HAVING COUNT(DISTINCT AttendanceType) = 2
        ),
        attendance_ggm AS (
            SELECT 
                UserID,
                DATE(AttendanceTime) AS att_date
            FROM CamsBiometricAttendance_GGM
            WHERE AttendanceType IN ('CheckIn', 'CheckOut')
            GROUP BY UserID, DATE(AttendanceTime)
            HAVING COUNT(DISTINCT AttendanceType) = 2
        ),
        present_days AS (
            SELECT 
                e.empname,
                e.UserID,
                CASE 
                    WHEN e.work_location = 'Visakhapatnam' THEN (
                        SELECT GROUP_CONCAT(DISTINCT a.att_date ORDER BY a.att_date)
                        FROM attendance_vskp a
                        WHERE a.UserID = e.UserID
                          AND a.att_date BETWEEN GREATEST(e.empdoj, '2024-01-01') AND '2025-03-31'
                    )
                    WHEN e.work_location = 'Gurugram' THEN (
                        SELECT GROUP_CONCAT(DISTINCT a.att_date ORDER BY a.att_date)
                        FROM attendance_ggm a
                        WHERE a.UserID = e.UserID
                          AND a.att_date BETWEEN GREATEST(e.empdoj, '2024-01-01') AND '2025-03-31'
                    )
                    ELSE ''
                END AS present_dates
            FROM emp e
            WHERE e.empname = 'PUTSALA HARSHA PRIYA'
        )
        SELECT 
            e.empname,
            e.UserID,
            p.present_dates
        FROM emp e
        LEFT JOIN present_days p ON e.UserID = p.UserID
        WHERE e.empname = 'PUTSALA HARSHA PRIYA';
        """

        cursor.execute(query)
        result = cursor.fetchall()

        conn.close()

        # Extracting the present dates from the result
        if result:
            present_days = result[0]['present_dates']
            present_days_list = present_days.split(',') if present_days else []

            # Creating a DataFrame from the present days
            df = pd.DataFrame(present_days_list, columns=['Present Days'])

            # Saving the DataFrame to an Excel file
            excel_file_path = "./present_days_putsala_harsha_priya.csv"
            df.to_csv(excel_file_path, index=False)

            return excel_file_path
        else:
            return "No data found for PUTSALA HARSHA PRIYA"

    except mysql.connector.Error as err:
        print(f"Database error: {err}")
        return None

# Fetch the present days and generate the Excel file
file_path = fetch_present_days()
file_path


'./present_days_putsala_harsha_priya.csv'

In [52]:
import pandas as pd
import mysql.connector

# Function to fetch only the working days (without redundancy)
def fetch_working_days():
    try:
        conn = connect_to_db()
        cursor = conn.cursor(dictionary=True)

        query = """
        WITH RECURSIVE all_dates AS (
            SELECT DATE('2024-01-01') AS dt
            UNION ALL
            SELECT dt + INTERVAL 1 DAY
            FROM all_dates
            WHERE dt + INTERVAL 1 DAY <= '2025-03-31'
        ),
        valid_dates AS (
            SELECT dt
            FROM all_dates
            WHERE DAYOFWEEK(dt) != 1  -- Exclude Sundays
            AND dt NOT IN (SELECT `date` FROM holiday)  -- Exclude holidays
        )
        SELECT dt AS working_day
        FROM valid_dates;
        """

        cursor.execute(query)
        result = cursor.fetchall()

        conn.close()

        # Create a DataFrame from the result
        df = pd.DataFrame(result)

        # Renaming the column for clarity
        df.columns = ['Working Days']

        # Saving the DataFrame to a CSV file
        csv_file_path = "working_days.csv"
        df.to_csv(csv_file_path, index=False)

        return csv_file_path

    except mysql.connector.Error as err:
        print(f"Database error: {err}")
        return pd.DataFrame()


# Fetch and print the result, then save to CSV
csv_file = fetch_working_days()
print(f"CSV file saved at: {csv_file}")


CSV file saved at: working_days.csv


In [53]:
df['attendance_percentage'] = (df['present_days'] / df['working_days']) * 100
df['attendance_percentage'] = df['attendance_percentage'].round(2)   # Correct rounding

def assign_points(percentage):
    if percentage >= 95:
        return 20
    elif 90 <= percentage < 95:
        return 16
    elif 85 <= percentage < 90:
        return 12
    elif 80 <= percentage < 85:
        return 8
    else:
        return 4 if percentage >= 0 else 0

df['points'] = df['attendance_percentage'].apply(assign_points)

print(df[['UserID', 'empname', 'attendance_percentage', 'points']].iloc[50:60])


KeyError: 'present_days'

In [54]:
df['UserID'] = df['UserID'].astype(int)
df_user_49 = df[df['UserID'] == 50]
print(df_user_49)


                empname  UserID  \
0  PUTSALA HARSHA PRIYA      50   

                                       present_dates  
0  2024-06-04,2024-06-05,2024-06-06,2024-06-07,20...  


In [55]:
import mysql.connector

def push_to_kpi_attendance(df):
    try:
        conn = connect_to_db()  # Reuse your existing connection function
        cursor = conn.cursor()

        # Optional: Drop the table if it already exists
        cursor.execute("DROP TABLE IF EXISTS kpi_attendance")

        # Create the new table
        create_table_query = """
        CREATE TABLE kpi_attendance (
            UserID INT,
            empname VARCHAR(255),
            attendance_percentage FLOAT,
            points INT
        )
        """
        cursor.execute(create_table_query)

        # Insert DataFrame records one by one
        insert_query = """
        INSERT INTO kpi_attendance (UserID, empname, attendance_percentage, points)
        VALUES (%s, %s, %s, %s)
        """
        for _, row in df.iterrows():
            cursor.execute(insert_query, (
                int(row['UserID']),
                row['empname'],
                float(row['attendance_percentage']),
                int(row['points'])
            ))

        conn.commit()
        print("Data pushed to kpi_attendance table successfully.")

    except mysql.connector.Error as err:
        print(f"Database error: {err}")
    finally:
        if conn.is_connected():
            cursor.close()
            conn.close()
push_to_kpi_attendance(df)

KeyError: 'attendance_percentage'

In [56]:
def fetch_leave_data(email):
    try:
        conn=connect_to_db()
        cursor=conn.cursor(dictionary=True)
        query="""
        SELECT l.id, lb.empname, lb.cl, lb.sl, lb.co, lb.empemail, l.hrremark, l.mgrremark, l.aprremark, l.from, l.to, l.desg
        FROM leavebalance lb
        JOIN leaves l ON lb.empemail = l.empemail
        WHERE lb.empemail = %s
        """
        cursor.execute(query,(email,))
        result=cursor.fetchall()
        conn.close()
        return pd.DataFrame(result) if result else None
    except mysql.connector.Error as err:
        print(f"Error:{err}")
        return None

In [57]:
#leave eligibility

def can_apply_leave(data):
    if data is not None and not data.empty:
        # Convert columns to numeric, handling non-numeric values
        data["cl"] = pd.to_numeric(data["cl"], errors='coerce').fillna(0)
        data["sl"] = pd.to_numeric(data["sl"], errors='coerce').fillna(0)
        data["co"] = pd.to_numeric(data["co"], errors='coerce').fillna(0)
        
        # Calculate total leave balance (lb = cl + sl + co)
        data["lb"] = data["cl"] + data["sl"] + data["co"]
        
        # Check if leave can be applied (i.e., if total balance > 0)
        data["can_apply_leave"] = data["lb"] > 0
        return data
    else:
        return "not having enough leave balance"

In [58]:
def fetch_leave_data(email):
    try:
        conn=connect_to_db()
        cursor=conn.cursor(dictionary=True)
        query="""
        SELECT lb.empname, lb.cl, lb.sl, lb.co, lb.empemail,lb.icl,lb.isl,lb.ico
        FROM leavebalance lb
        WHERE lb.empemail = %s
        """
        cursor.execute(query,(email,))
        result=cursor.fetchall()
        conn.close()
        return pd.DataFrame(result) if result else None
    except mysql.connector.Error as err:
        print(f"Error:{err}")
        return None
fetch_leave_data(email="putsalaharshapriya@gmail.com")

,empname,cl,sl,co,empemail,icl,isl,ico
0,PUTSALA HARSHA PRIYA,1,7,0,putsalaharshapriya@gmail.com,10,10,0


In [59]:
import mysql.connector
import pandas as pd

def connect_to_db():
    return mysql.connector.connect(
        host="localhost",
        user="root",
        password="",
        database="ems"
    )

def assign_lop_score(lop_balance):
    lop_days = abs(lop_balance)

    if lop_days == 0:
        return 15
    elif lop_days <= 1:
        return 13
    elif lop_days <= 2:
        return 11
    elif lop_days <= 3:
        return 9
    elif lop_days <= 5:
        return 6
    elif lop_days <= 7:
        return 3
    elif lop_days <=10:     
        return 1
    else:
        return 0

def fetch_lop_balance(email=None):
    try:
        conn = connect_to_db()
        cursor = conn.cursor(dictionary=True)

        if email:
            query = """
            SELECT lb.id, lb.empname, lb.cl, lb.sl, lb.co, lb.empemail, lb.icl, lb.isl, lb.ico
            FROM leavebalance lb
            WHERE lb.empemail = %s
            """
            cursor.execute(query, (email,))
        else:
            query = """
            SELECT lb.id, lb.empname, lb.cl, lb.sl, lb.co, lb.empemail, lb.icl, lb.isl, lb.ico
            FROM leavebalance lb
            """
            cursor.execute(query)

        result = cursor.fetchall()
        conn.close()

        if result:
            df = pd.DataFrame(result)
            numeric_columns = ['cl', 'sl', 'co']
            for col in numeric_columns:
                df[col] = pd.to_numeric(df[col], errors='coerce')
            df['total'] = df['cl'] + df['sl'] + df['co']
            df['lop_balance'] = df['total'].apply(lambda x: x if x < 0 else 0)
            df['lop_score'] = df['lop_balance'].apply(assign_lop_score)

            return df[['id', 'empname', 'lop_balance', 'lop_score']]
        else:
            return pd.DataFrame(columns=['id', 'empname', 'lop_balance', 'lop_score'])

    except mysql.connector.Error as err:
        print(f"Error: {err}")
        return None

lop_df = fetch_lop_balance()
print(lop_df.head(25))


    id                           empname  lop_balance  lop_score
0    3       YERRAMSETTI SUSANTHA SANKAR          0.0         15
1    6                 YATHIRAJAM SOWMYA          0.0         15
2    7                BAGADI PRAKASH RAO          0.0         15
3    8                GANDI MANOHARA RAO          0.0         15
4    9                    THUMPALA ANAND          0.0         15
5   10        PILLA INDIRAVATHI SOWJANYA          0.0         15
6   11  NARADA MOHAN NANDA KISHORE REDDY          0.0         15
7   12               ABHILASH KUMAR SAHU          0.0         15
8   13                ADAPA CHANDRA RAMU          0.0         15
9   14                       SANDIP BERA          0.0         15
10  15                PEDDINTI VENKATESH          0.0         15
11  16                     BONDI MANOHAR          0.0         15
12  17                    KAJAL MAJUMDAR         -4.0          6
13  18                RAJESH MIDITHIPATI          0.0         15
14  19              NAGAB

In [60]:
import mysql.connector
import pandas as pd

def connect_to_db():
    return mysql.connector.connect(
        host="localhost",
        user="root",
        password="",
        database="ems"
    )

def assign_lop_score(lop_balance):
    lop_days = abs(lop_balance)

    if lop_days == 0:
        return 15
    elif lop_days <= 1:
        return 13
    elif lop_days <= 2:
        return 11
    elif lop_days <= 3:
        return 9
    elif lop_days <= 5:
        return 6
    elif lop_days <= 7:
        return 3
    elif lop_days <= 10:     
        return 1
    else:
        return 0

def fetch_lop_balance():
    try:
        conn = connect_to_db()
        cursor = conn.cursor(dictionary=True)

        query = """
            SELECT pl.empname, SUM(pl.flop) AS total_flop
            FROM payroll_lop pl
            GROUP BY pl.empname
        """
        cursor.execute(query)

        result = cursor.fetchall()
        conn.close()

        if result:
            df = pd.DataFrame(result)
            df['total_flop'] = pd.to_numeric(df['total_flop'], errors='coerce').fillna(0)
            df['lop_balance'] = df['total_flop']  # Directly total_flop
            df['lop_score'] = df['lop_balance'].apply(assign_lop_score)

            return df[['empname', 'lop_balance', 'lop_score']]
        else:
            return pd.DataFrame(columns=['empname', 'lop_balance', 'lop_score'])

    except mysql.connector.Error as err:
        print(f"Error: {err}")
        return None

lop_df = fetch_lop_balance()
print(lop_df.head(25))


                  empname  lop_balance  lop_score
0           AMIT KHANDURI          0.0         15
1          ARSHPREET KAUR          6.0          3
2           BADI SIMHADRI          3.5          6
3           BONDI MANOHAR          2.0         11
4          DATTI SUDHAKAR         10.0          1
5         DWARAPUDI SRINU          5.0          6
6        FULKURIA INDOWAR          0.0         15
7      GANDI MANOHARA RAO          1.0         13
8          JITENDER KUMAR          0.0         15
9          KAJAL MAJUMDAR         23.5          0
10  KARRI DIVYA MANIKANTA          0.5         13
11  KITLANGI TABITHA ROJA          9.5          1
12         KUNDETI NANAJI          4.0          6
13        KUNDRAPU MURALI         10.5          0
14         MANPREET SINGH         12.0          0
15    NAKKA ARAVIND REDDY          1.0         13
16        NALLA VARALAXMI          7.0          3
17      NAMMI SATHYAVATHI          1.0         13
18         PADDA NAGARAJU          8.5          1


In [61]:
import mysql.connector

def connect_to_db():
    return mysql.connector.connect(
        host="localhost",
        user="root",
        password="",
        database="ems"
    )

def push_to_kpi_lop(df):
    try:
        conn = connect_to_db()  
        cursor = conn.cursor()

        cursor.execute("DROP TABLE IF EXISTS kpi_lop")

        create_table_query = """
        CREATE TABLE kpi_lop (
            id INT AUTO_INCREMENT PRIMARY KEY,
            empname VARCHAR(255),
            lop_balance FLOAT,
            points INT
        )
        """
        cursor.execute(create_table_query)

        insert_query = """
        INSERT INTO kpi_lop (empname, lop_balance, points)
        VALUES (%s, %s, %s)
        """
        for _, row in df.iterrows():
            cursor.execute(insert_query, (
                row['empname'],
                float(row['lop_balance']),
                int(row['lop_score'])
            ))

        conn.commit()
        print("Data pushed to kpi_lop table successfully.")

    except mysql.connector.Error as err:
        print(f"Database error: {err}")
    finally:
        if conn.is_connected():
            cursor.close()
            conn.close()

# Now call the function
push_to_kpi_lop(lop_df)


Data pushed to kpi_lop table successfully.


In [62]:
def late_days():
    try:
        conn = connect_to_db()
        cursor = conn.cursor(dictionary=True)

        query = """
           SELECT 
            e.UserID,
            e.empname,
            e.empdoj,
            COUNT(*) AS late_days
        FROM (
            SELECT 
                c.UserID,
                DATE(c.AttendanceTime) AS att_date
            FROM 
                CamsBiometricAttendance c
            WHERE 
                c.AttendanceType = 'CheckIn'
                AND TIME(c.AttendanceTime) > '09:30:00'
            GROUP BY 
                c.UserID, DATE(c.AttendanceTime)
        ) late_checkins
        JOIN emp e ON e.UserID = late_checkins.UserID
        WHERE 
            late_checkins.att_date >= e.empdoj
            AND late_checkins.att_date <= CURDATE()
        GROUP BY 
            e.UserID, e.empname, e.empdoj;

        """
        cursor.execute(query)
        result = cursor.fetchall()

        # Close the connection
        conn.close()
        
        df = pd.DataFrame(result)

        return df

    except mysql.connector.Error as err:
        print(f"Database error: {err}")
        return []

late_days()

,UserID,empname,empdoj,late_days
0,1,YATHIRAJAM SOWMYA,2022-10-01,293
1,11,ARSHPREET KAUR,2024-07-29,160
2,11,SANDIP BERA,2023-07-25,258
3,12,GANDI MANOHARA RAO,2022-11-01,31
4,13,SHANKAR,2024-07-30,146
...,...,...,...,...
67,7,RAHUL,2024-06-10,130
68,8,BAGADI PRAKASH RAO,2022-10-01,108
69,8,FULKURIA INDOWAR,2024-06-03,105
70,9,DEEPAK YADAV,2024-07-13,151


In [66]:
df = late_days()

df['UserID'] = pd.to_numeric(df['UserID'], errors='coerce').astype('Int64')

specific_user_id = 60
user_row = df[df['UserID'] == specific_user_id]

print(f"Details for UserID {specific_user_id}:\n", user_row)


Details for UserID 60:
     UserID                  empname      empdoj  late_days
63      60  DHARMALA JOSHITHA REDDY  2024-11-20         69


In [ ]:
import pandas as pd
import mysql.connector

def percent_points(percent):
    if percent <= 30:
        return 10
    elif percent <= 40:
        return 9
    elif percent <= 50:
        return 8
    elif percent <= 60:
        return 7
    elif percent <= 70:
        return 6
    elif percent <= 80:
        return 5
    elif percent <= 90:
        return 4
    elif percent <= 95:
        return 3
    else:
        return 2

def get_late_and_present_days():
    try:
        conn = connect_to_db()
        cursor = conn.cursor(dictionary=True)

        # Query for late days
        late_query = """
            SELECT 
                e.empname,
                e.empdoj,
                COUNT(*) AS late_days
            FROM (
                SELECT 
                    c.UserID,
                    DATE(c.AttendanceTime) AS att_date
                FROM 
                    CamsBiometricAttendance c
                WHERE 
                    c.AttendanceType = 'CheckIn'
                    AND TIME(c.AttendanceTime) > '09:30:00'
                GROUP BY 
                    c.UserID, DATE(c.AttendanceTime)
            ) late_checkins
            JOIN emp e ON e.UserID = late_checkins.UserID
            WHERE 
                late_checkins.att_date >= e.empdoj
                AND late_checkins.att_date <= CURDATE()
            GROUP BY 
                e.empname, e.empdoj;
        """
        cursor.execute(late_query)
        late_result = cursor.fetchall()
        late_df = pd.DataFrame(late_result)

        # Query for present days
        present_query = """
            SELECT 
                e.empname,
                e.empdoj,
                COUNT(*) AS present_days
            FROM (
                SELECT 
                    c.UserID,
                    DATE(c.AttendanceTime) AS att_date
                FROM 
                    CamsBiometricAttendance c
                WHERE 
                    c.AttendanceType IN ('CheckIn', 'CheckOut')
                GROUP BY 
                    c.UserID, DATE(c.AttendanceTime)
                HAVING 
                    COUNT(DISTINCT c.AttendanceType) = 2
            ) valid_days
            JOIN emp e ON e.UserID = valid_days.UserID
            WHERE 
                valid_days.att_date >= e.empdoj
                AND valid_days.att_date <= CURDATE()
            GROUP BY 
                e.empname, e.empdoj;
        """
        cursor.execute(present_query)
        present_result = cursor.fetchall()
        present_df = pd.DataFrame(present_result)
        conn.close()

        final_df = pd.merge(present_df, late_df, on=['empname', 'empdoj'], how='outer')
        #points for late
        final_df['late_days'] = final_df['late_days'].fillna(0).astype(int)
        final_df['present_days'] = final_df['present_days'].fillna(0).astype(int)
        final_df['late_percent'] = ((final_df['late_days']) / final_df['present_days']) * 100
        final_df['late_percent'] = final_df['late_percent'].fillna(0).round(2)
        final_df['points'] = final_df['late_percent'].apply(percent_points)

        return final_df

    except mysql.connector.Error as err:
        print(f"Database error: {err}")
        return pd.DataFrame()

df = get_late_and_present_days()
print(df.head(10))


               empname      empdoj  present_days  late_days  late_percent  \
0  AATMAKURI MANIKANTA  2024-06-04           223         79         35.43   
1           ABDUL NABI  2024-08-28           153         93         60.78   
2   ADAPA CHANDRA RAMU  2023-03-11           255        217         85.10   
3      AKANSHA PANDEY   2024-06-01           200        174         87.00   
4       ARSHPREET KAUR  2024-07-29           179        160         89.39   
5           ARUN KUMAR  2024-12-27            65         58         89.23   
6        BADI SIMHADRI  2024-09-12           143         73         51.05   
7   BAGADI PRAKASH RAO  2022-10-01           169        108         63.91   
8        BONDI MANOHAR  2023-09-11           325        183         56.31   
9          DADI PRASAD  2023-03-11           332        184         55.42   

   points  
0       9  
1       6  
2       4  
3       4  
4       4  
5       4  
6       7  
7       6  
8       7  
9       7  


In [ ]:
import mysql.connector
import pandas as pd


def assign_manager_score(score):
    if score == 25:
        return 10
    elif score >= 22:
        return 9
    elif score >= 19:
        return 8
    elif score >= 16:
        return 7
    elif score >= 13:
        return 6
    elif score >= 10:
        return 5
    elif score >= 7:
        return 4
    elif score >= 4:
        return 3
    elif score >= 2:
        return 2
    elif score == 1:
        return 1
    else:
        return 0

def fetch_manager_feedback_scores():
    try:
        conn = connect_to_db()
        cursor = conn.cursor(dictionary=True)

        query = """
            SELECT id, empname, score
            FROM manager_feedback
        """
        cursor.execute(query)
        result = cursor.fetchall()
        conn.close()

        if result:
            df = pd.DataFrame(result)
            df['score'] = pd.to_numeric(df['score'], errors='coerce')
            df['manager_score'] = df['score'].apply(assign_manager_score)
            return df[['id', 'empname', 'manager_score']]
        else:
            return pd.DataFrame(columns=['id', 'empname', 'manager_score'])

    except mysql.connector.Error as err:
        print(f"Error: {err}")
        return pd.DataFrame(columns=['id', 'empname', 'manager_score'])

manager_df = fetch_manager_feedback_scores()
print(manager_df.head(25))


    id                           empname  manager_score
0    1  NARADA MOHAN NANDA KISHORE REDDY              7
1    2                     BONDI MANOHAR              6
2    3                BAGADI PRAKASH RAO              9
3    4                 YATHIRAJAM SOWMYA              6
4    5       YERRAMSETTI SUSANTHA SANKAR              2
5    6               NAKKA ARAVIND REDDY              4
6    7             GORAPALLI MEGHA SHYAM              1
7    8                 KUNDRAPU SRINIVAS              6
8    9                  SURAMPUDI LAXMAN              7
9   10               INDALA SATISH KUMAR              8
10  11              PUTSALA HARSHA PRIYA             10
11  12                   GUJJI VARSHITHA              0
12  13           DHARMALA JOSHITHA REDDY             10
